In [1]:
import numpy     as np
from scipy.linalg import svd, qr
from scipy.linalg import qr, svd, toeplitz, eigh, norm
from scipy.spatial.distance import squareform, pdist
from sklearn.cluster import SpectralClustering, KMeans
from sklearn.decomposition import PCA
from scipy.optimize import linear_sum_assignment
from scipy.io import wavfile
from sklearn.metrics import confusion_matrix, adjusted_rand_score

import tempfile

import scipy.signal as signal
from IPython.display import Audio
import io

import itertools

import param
import colorcet as cc

import panel     as pn; pn.extension('mathjax', 'katex')
import holoviews as hv; hv.extension( "bokeh", logo=False)
import plotly.graph_objects as go

import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", message="k >= N for N * N square matrix")

<div style="height:2cm;">
<div style="float:center;width:100%;text-align:center;"><strong style="height:100px;color:darkred;font-size:40px;">Applications of the Grassmannian</strong>
</div></div>

# 1. Introduction

## 1.1. Objective

The previous notebooks introduced the Grassmannian manifold $\mathrm{Gr}(k, n)$,<br>
the space of all $k$-dimensional subspaces of $\mathbb{R}^n$.<br>
We developed the necessary tools: **principal angles, Grassmannian distances, geodesics**,<br>
and **optimization techniques** constrained to the manifold.

This notebook considers applications in which subspaces are the primary data objects.<br>
These include problems where information is distributed across sets of vectors,<br>
or where system states evolve within a low-dimensional linear structure.

Relevant contexts include:
- **Computer vision:** modeling image sets by low-rank approximations
- **Signal processing:** channel representations in MIMO systems
- **Model reduction:** subspace dynamics in large-scale simulations
- **Machine learning:** classification and clustering in subspace-based settings

In such cases, the Grassmannian framework supports tasks like interpolation, averaging, and optimization over subspaces.

We assume familiarity with the material developed in:
- [**PrincipalAngles.ipynb**](PrincipalAngles.ipynb)
- [**GrassmannianIntro.ipynb**](GrassmannianIntro.ipynb)
- [**GrassmannianGeodesicsOptimization.ipynb**](GrassmannianGeodesicsOptimization.ipynb)

The present notebook focuses on how the Grassmannian arises in practice, and how its geometry can be applied in computation.

We begin by reviewing why subspace-based models are natural in many applications.

## 1.2 Subspaces as Fundamental Representations

Many data- and model-driven problems lead not to individual vectors, but to sets of vectors that share a linear structure.<br>
It is often more appropriate to describe such a set by the subspace it spans,<br>
rather than by any particular basis or representative.

* In image recognition, for example, a set of images of a single subject under varying lighting<br>
can be approximated by a low-dimensional subspace.<br>
* In signal processing, time-varying observations may evolve within a subspace.
* In model reduction, dominant modes are extracted from snapshots of a high-dimensional system<br> and used to define a reduced basis.

In each case, the subspace — not the data points — becomes the object of interest.<br>
Comparisons, interpolations, and optimizations must be carried out over subspaces.<br>
This naturally leads to viewing subspaces as points on the Grassmann manifold $\mathrm{Gr}(k, n)$.

We will now recall the basic geometry of $\mathrm{Gr}(k, n)$ needed for applications.

____

The Grassmannian $\mathrm{Gr}(k, n)$ is a smooth manifold whose points correspond to $k$-dimensional linear subspaces of $\mathbb{R}^n$.<br>
$\qquad$ For computational purposes, a point in $\mathrm{Gr}(k, n)$ is typically represented<br>
$\qquad$ by an $n \times k$ matrix with orthonormal columns.<br>
$\qquad$ Two such matrices $U$ and $UQ$ (with $Q \in O(k)$) represent the same subspace.

Distances between subspaces are computed via the principal angles $\theta_1, \dots, \theta_k$ between them.<br>
$\qquad$ These are obtained from the singular values of $U^\top V$ for two orthonormal bases $U, V \in \mathbb{R}^{n \times k}$.

Standard metrics include:
- **Geodesic (arc length) distance:**
  $\quad d_g(U, V) = \left( \sum_{i=1}^k \theta_i^2 \right)^{1/2} $
- **Chordal distance**:
  $\qquad\qquad\qquad\quad d_c(U, V) = \left( \sum_{i=1}^k \sin^2 \theta_i \right)^{1/2} $
- **Projection (Frobenius) distance:**
  $\quad d_p(U, V) = \Vert U U^T - V V^T \Vert_F $

These distances define different geometries but are all intrinsic to the Grassmannian structure.

Geodesics on $\mathrm{Gr}(k, n)$ can be computed explicitly.<br>
$\qquad$ Given two points $U$ and $V$ in $\mathrm{Gr}(k, n)$, a shortest path connecting them<br>
$\qquad$ can be parameterized using the principal vectors and angles.<br>
$\qquad$ This structure supports interpolation, averaging, and optimization directly on the manifold.<br>

The tools developed in earlier notebooks allow us to compute these distances and geodesics numerically.<br>
$\qquad$ What follows is a series of applications where these structures are central.


# 2. Subspace Extraction from Data

**Many high-dimensional datasets are composed of samples that approximately lie on a union of low-dimensional subspaces.**<br>
In such cases, classical clustering techniques like $k$-means, which assume isotropic point clouds,<br>
often fail to capture the underlying structure.

The goal of **subspace clustering** is to **partition the data into groups, each associated with a low-dimensional subspace.**<br>
Once the subspaces are estimated, each group can be represented as a point on $\mathrm{Gr}(k, n)$,<br>
and the clustering task becomes one of grouping points on the Grassmannian.

Typical applications include:
- Face recognition under varying illumination
- Motion segmentation in videos
- Multivariate time series with regime switching

This section demonstrates how Grassmannian distances can be used to build an affinity matrix<br>
and perform clustering based on subspace geometry.

____

#### Problem Statement

Let $X = \{x_i\}_{i=1}^{L M} \subset \mathbb{R}^N$ be a dataset consisting of $L$ clusters of $M$ points each, in ambient dimension $N$.<br>$\qquad$
Each cluster lies near an unknown $K$-dimensional linear subspace

$\qquad
X = \bigcup_{\ell=1}^L \mathcal{S}_\ell + \text{noise}, \quad \mathcal{S}_\ell \in \mathrm{Gr}(K, N)
$

The task is to assign each $x_i$ to its generating subspace $\mathcal{S}_\ell$, without access to the subspaces or ground-truth labels.

When $K \ll N$, Euclidean distance is ineffective: points from the same subspace may be far apart,<br>$\qquad$
and subspaces may be nearly orthogonal. Subspace-based clustering overcomes this by reasoning geometrically in $\mathrm{Gr}(K, N)$.

In [13]:
hv.extension('plotly', logo=False)

class SubspaceGeometryDemo(pn.viewable.Viewer):
    spread_factor      = param.Number(1.0, bounds=(0.01, 5.0), step=0.01, label="Spread Factor (Right Panel)")
    points_per_cluster = param.Integer(50, bounds=(10, 200), label="Points per Cluster")
    cluster_offset     = param.Number(3.0, bounds=(0.0, 10.0), step=0.1, label="Cluster Center Offset (Right Panel)")
    show_clusters      = param.Boolean(False, label="Show k-means result")

    def __init__(self, **params):
        super().__init__(**params)
        self._generate_fixed_subspaces()
        self._sample_left_panel_points()

    def _generate_fixed_subspaces(self):
        self.B1 = np.eye(3)[:, :2]  # XY plane
        angle_rad = np.radians(50)
        R = np.array([
            [1, 0, 0],
            [0, np.cos(angle_rad), -np.sin(angle_rad)],
            [0, np.sin(angle_rad), np.cos(angle_rad)]
        ])
        self.B2 = R @ self.B1

    def _sample_points(self, B, spread, center=np.zeros(3)):
        coeffs = np.random.randn(2, self.points_per_cluster)
        X = B @ coeffs + spread * np.random.randn(3, self.points_per_cluster)
        X += center[:, np.newaxis]
        return X.T

    def _sample_left_panel_points(self):
        spread_left = 0.05
        offset_left = np.array([4.0, 0.0, 0.0])  # move cluster 2 in X direction
        self.X1_left = self._sample_points(self.B1, spread_left, center=np.zeros(3))
        self.X2_left = self._sample_points(self.B2, spread_left, center=offset_left)

    def _create_plane_mesh(self, B, scale=3.0, opacity=0.6, color='lightgray'):
        u = np.linspace(-scale, scale, 10)
        v = np.linspace(-scale, scale, 10)
        uu, vv = np.meshgrid(u, v)
        plane_points = np.outer(B[:, 0], uu.flatten()) + np.outer(B[:, 1], vv.flatten())
        X = plane_points[0, :].reshape(uu.shape)
        Y = plane_points[1, :].reshape(uu.shape)
        Z = plane_points[2, :].reshape(uu.shape)

        return go.Surface(
            x=X, y=Y, z=Z,
            showscale=False,
            opacity=opacity,
            colorscale=[[0, color], [1, color]],
            hoverinfo='skip'
        )

    def _create_plot(self, X1, X2, height=350, show_kmeans=False):
        fig = go.Figure()
        fig.add_trace(self._create_plane_mesh(self.B1))
        fig.add_trace(self._create_plane_mesh(self.B2))

        X_all = np.vstack([X1, X2])
        if show_kmeans:
            kmeans = KMeans(n_clusters=2, n_init='auto').fit(X_all)
            colors = ['blue', 'red']
            for i in range(2):
                cluster_points = X_all[kmeans.labels_ == i]
                fig.add_trace(go.Scatter3d(
                    x=cluster_points[:, 0], y=cluster_points[:, 1], z=cluster_points[:, 2],
                    mode='markers',
                    marker=dict(size=4, color=colors[i]),
                    name=f'k-means Cluster {i+1}'
                ))
        else:
            fig.add_trace(go.Scatter3d(
                x=X1[:, 0], y=X1[:, 1], z=X1[:, 2],
                mode='markers',
                marker=dict(size=4, color='blue'),
                name='Cluster 1'
            ))
            fig.add_trace(go.Scatter3d(
                x=X2[:, 0], y=X2[:, 1], z=X2[:, 2],
                mode='markers',
                marker=dict(size=4, color='red'),
                name='Cluster 2'
            ))

        fig.update_layout(
            scene=dict(
                xaxis=dict(title='X', range=[-5, 5]),
                yaxis=dict(title='Y', range=[-5, 5]),
                zaxis=dict(title='Z', range=[-5, 5]),
                aspectmode='cube',
            ),
            margin=dict(l=10, r=10, t=30, b=10),
            autosize=False,
            height=height,
            width=400,
            legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='center', x=0.5)
        )
        return fig

    @param.depends('spread_factor', 'points_per_cluster', 'cluster_offset', 'show_clusters')
    def _right_panel_plot(self):
        offset_right = np.array([self.cluster_offset, 0.0, 0.0])
        X1_right = self._sample_points(self.B1, self.spread_factor, center=np.zeros(3))
        X2_right = self._sample_points(self.B2, self.spread_factor, center=offset_right)
        fig_right = self._create_plot(X1_right, X2_right, show_kmeans=self.show_clusters)
        return pn.pane.Plotly(fig_right, config={'displayModeBar': False}, height=350, width=400)

    def __panel__(self):
        fig_left = self._create_plot(self.X1_left, self.X2_left, show_kmeans=False)
        left_pane = pn.pane.Plotly(fig_left, config={'displayModeBar': False}, height=350, width=400)

        right_pane = pn.panel(self._right_panel_plot, height=350, width=400)

        caption = pn.pane.Markdown(r"""
### Subspace Geometry Demo
Two 2D subspaces (planes) in $$\mathbb{R}^3$$, with sample points drawn near each plane.
- **Left panel**: simple example — two well-separated clusters (offset centers). Standard clustering methods (e.g. k-means) succeed.
- **Right panel**: spread controlled by slider; cluster center offset controlled by slider:
    - As spread increases and offset decreases, **clusters begin to overlap in Euclidean space**.
    - **Standard clustering begins to fail**.
    - **Subspace-aware methods are required** to correctly identify the structure.

Both panels use the **same fixed subspaces** for comparison.
""", width=400)

        return pn.Column(
            pn.Row(left_pane, pn.Spacer(width=100), right_pane, sizing_mode='fixed', width=600),
            pn.Row(
                caption,
                pn.Spacer(width=50),
                pn.Column(
                    pn.Spacer(height=30),
                    pn.Param(self, parameters=['spread_factor', 'points_per_cluster', 'cluster_offset', 'show_clusters'], show_name=False, width=300)
                )
            )
        )

SubspaceGeometryDemo().servable()

Column
    [0] Row(sizing_mode='fixed', width=600)
        [0] Plotly(Figure, config={'displayModeBar': False}, height=350, sizing_mode='fixed', width=400)
        [1] Spacer(width=100)
        [2] ParamMethod(method, _pane=Plotly, defer_load=False, height=350, sizing_mode='fixed', width=400)
    [1] Row
        [0] Markdown(str, width=400)
        [1] Spacer(width=50)
        [2] Column
            [0] Spacer(height=30)
            [1] Param(SubspaceGeometryDemo, show_name=False, width=300)

____
In $\mathbb{R}^N$, data sampled near distinct $K$-dimensional subspaces can appear highly entangled<br>
under Euclidean distance when noise or spread is large.<br>
However, each **local neighborhood retains an approximate tangent space** aligned with its true subspace.

By partitioning the data into overlapping groups and extracting these local tangent spaces via rank-$K$ SVD,<br>
we represent each neighborhood by a point on the Grassmannian $\mathrm{Gr}(K, N)$.<br>
Distances on $\mathrm{Gr}(K, N)$ capture intrinsic subspace similarity, enabling construction of an affinity matrix<br>
that reflects true cluster structure.

Spectral clustering on this graph thus succeeds where point-based methods fail.

____
The method proceeds as follows:
1. Partition $X$ into overlapping local groups of size $m$.
2. For each group, form a matrix $X \in \mathbb{R}^{N \times m}$ and compute the rank-$K$ truncated SVD:<br>
   $\qquad
   X = U \Sigma V^\top, \quad U \in \mathbb{R}^{N \times K}
   $<br>
   The estimated subspace is $\mathscr{C}(U)$.
3. Represent each subspace by its orthonormal basis $U$.
4. Compute pairwise distances between subspaces using principal angles<br>
   $\qquad
   d(U_i, U_j) = \left\Vert \theta(U_i, U_j) \right\Vert_2, \quad \theta = \text{principal angles}
   $
5. Construct a symmetric affinity matrix $A \in \mathbb{R}^{n \times n}$ using a Gaussian kernel applied to the Grassmannian distances<br>
   $\qquad
   A_{ij} = \exp\left( -\frac{d(U_i, U_j)^2}{2\sigma^2} \right)
   $<br>
   where $\sigma$ controls the sensitivity to distance.

Define the (unnormalized) graph Laplacian $L = D - A$, where $D$ is the diagonal matrix with $D_{ii} = \sum_j A_{ij}$.<br>
$\qquad$ Spectral clustering embeds the data using the first $L$ eigenvectors of $L$, followed by $k$-means.

When subspaces are well-separated ($K \ll N$) and noise is small, clustering accuracy is high.<br>
$\qquad$ In the demo below, correct assignments appear as colored circles; misclassified subspaces are marked with crosses.<br>
$\qquad$ You can vary $N$, $K$, $L$, $M$, and noise level to explore different regimes.

In [17]:
hv.extension('bokeh', logo=False)

def permute_labels(true_labels, pred_labels):
    C = confusion_matrix(true_labels, pred_labels)
    row_ind, col_ind = linear_sum_assignment(-C)
    mapping = dict(zip(col_ind, row_ind))
    return np.array([mapping.get(p, -1) for p in pred_labels])

class SubspaceClusteringDemo(pn.viewable.Viewer):
    n = param.Integer(20, bounds=(3, 50), label="Ambient Dimension (N)")
    k = param.Integer(3, bounds=(1, 20), label = "Subspace Dimension (K)")
    clusters = param.Integer(3, bounds=(2, 10), label="Clusters (L)")
    points_per_cluster = param.Integer(20, bounds=(5, 100), label="Points per Cluster (M)")
    groups_per_cluster = param.Integer(3, bounds=(1, 10), label="Groups per cluster")
    noise = param.Number(0.05, bounds=(0, 0.5), step=0.01)

    def __init__(self, **params):
        super().__init__(**params)
        self._enforce_valid_k()

    @pn.depends('n', watch=True)
    def _enforce_valid_k(self):
        self.param['k'].bounds = (1, self.n)
        if self.k > self.n:
            self.k = self.n

    def random_subspace(self, n, k):
        Q, _ = qr(np.random.randn(n, k), mode='economic')
        return Q

    def principal_angles(self, U, V):
        if U.shape != V.shape:
            raise ValueError("Subspace shapes are incompatible.")
        _, s, _ = svd(U.T @ V)
        s = np.clip(s, -1.0, 1.0)
        return np.arccos(s)

    def grassmannian_distance(self, U, V):
        try:
            theta = self.principal_angles(U, V)
            return np.linalg.norm(theta)
        except Exception:
            return np.inf

    def permute_labels(self, true_labels, pred_labels):
        C = confusion_matrix(true_labels, pred_labels)
        row_ind, col_ind = linear_sum_assignment(-C)
        mapping = dict(zip(col_ind, row_ind))
        return np.array([mapping.get(p, -1) for p in pred_labels])

    @param.depends('n', 'k', 'clusters', 'points_per_cluster', 'groups_per_cluster', 'noise')
    def view(self):
        n, k, L, M = self.n, self.k, self.clusters, self.points_per_cluster
        G = self.groups_per_cluster
        noise = self.noise

        rng = np.random.default_rng(42)
        true_bases = [self.random_subspace(n, k) for _ in range(L)]

        # Generate full dataset
        data = []
        labels = []
        for i, B in enumerate(true_bases):
            coeffs = rng.standard_normal((k, M))
            X = B @ coeffs + noise * rng.standard_normal((n, M))
            data.append(X.T)
            labels.extend([i] * M)

        data = np.vstack(data)
        labels = np.array(labels)

        # Estimate subspaces
        est_bases = []
        est_labels = []

        for i in range(L):
            start = i * M
            end = (i + 1) * M
            X_cluster = data[start:end]
            cluster_label = i
            indices = np.array_split(np.arange(M), G)
            for idx in indices:
                if len(idx) < k:
                    continue
                block = X_cluster[idx]
                X_block = block.T - block.T.mean(axis=1, keepdims=True)
                U, S, _ = svd(X_block, full_matrices=False)
                if S[k-1] > 1e-10:
                    est_bases.append(U[:, :k])
                    est_labels.append(cluster_label)

        if len(est_bases) < L:
            return hv.Points([]).opts(
                title="Too few estimated subspaces for clustering",
                height=400, width=500
            )

        # Pairwise distances
        m = len(est_bases)
        D = np.zeros((m, m))
        for i in range(m):
            for j in range(i + 1, m):
                d = self.grassmannian_distance(est_bases[i], est_bases[j])
                D[i, j] = D[j, i] = d

        if np.isinf(D).any():
            return hv.Points([]).opts(
                title="Distance computation failed",
                height=400, width=500
            )

        # Clustering
        sigma = 1.0
        A = np.exp(-D**2 / (2 * sigma**2))
        try:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", ConvergenceWarning)
                pred_labels = SpectralClustering(
                    n_clusters=L,
                    affinity='precomputed',
                    random_state=0
                ).fit_predict(A)
        except Exception:
            return hv.Points([]).opts(
                title="Clustering failed",
                height=400, width=500
            )

        # Label alignment + ARI
        aligned_pred = self.permute_labels(est_labels, pred_labels)
        ari = adjusted_rand_score(est_labels, aligned_pred)
        correct = (aligned_pred == np.array(est_labels))

        # Visualization
        X_flat = np.vstack([B[:, 0] for B in est_bases])
        X_proj = PCA(n_components=2).fit_transform(X_flat)
        coords = X_proj[:, 0], X_proj[:, 1]
        palette = cc.glasbey[:L]

        correct_pts = hv.Points(
            (coords[0][correct], coords[1][correct], aligned_pred[correct]),
            ['x', 'y'], 'Cluster'
        ).opts(marker='circle', color='Cluster', cmap=palette, size=6, tools=['hover'])

        incorrect_pts = hv.Points(
            (coords[0][~correct], coords[1][~correct], aligned_pred[~correct]),
            ['x', 'y'], 'Cluster'
        ).opts(marker='x', color='Cluster', cmap=palette, size=8, tools=['hover'])

        return (correct_pts * incorrect_pts).opts(
            width=500, height=400,
            title=f"Subspace clustering (ARI = {ari:.2f})"
        )

    def __panel__(self):
        widgets = pn.Param(
            self,
            parameters=['n', 'k', 'clusters', 'points_per_cluster', 'groups_per_cluster', 'noise'],
            show_name=False, width=300
        )
        caption = pn.pane.Markdown(
            "**Each point represents an estimated subspace,**<br> visualized by its first basis vector (via PCA) projected into 2D.<br>"
            "`circle` = correctly clustered, `x` = misclassified (after optimal label alignment).<br>"
            "**ARI** measures clustering accuracy up to label permutation; 1.0 is perfect.",
            width=500, height=50
        )
        return pn.Column(
            pn.Row(pn.Column(widgets, caption), self.view),
        )

SubspaceClusteringDemo().servable()

Column
    [0] Row
        [0] Column
            [0] Param(SubspaceClusteringDemo, show_name=False, width=300)
            [1] Markdown(str, height=50, sizing_mode='fixed', width=500)
        [1] ParamMethod(method, _pane=HoloViews, defer_load=False)

### Subspace Clustering Demo

This demo illustrates clustering based on Grassmannian distances between local subspace estimates.<br>
Each point corresponds to a local subspace, computed as the rank-$K$ SVD of a neighborhood of data points.<br>
Subspaces are represented by their first basis vectors and visualized in 2D via PCA.
- Circles: correctly classified subspaces (after optimal label alignment).
- X marks: misclassified subspaces.

Subspaces from the same ground-truth cluster should form distinct groups in the embedding. Misclassifications indicate overlap or estimation error.

---

**Expected behavior as parameters vary:**

- $K \ll N$, low noise: subspaces are well-separated; clustering is accurate.
- Increasing $K$ or decreasing $N$: subspaces overlap; principal angles shrink; clustering accuracy degrades.
- Increasing noise: local subspace estimates degrade; misclassifications increase.
- Group size $m \geq K+1$: local tangent spaces are well estimated; clustering is stable.
- Larger group sizes: neighborhoods span multiple subspaces; tangent space estimates are corrupted.

---

**Recommendation:** use $K \ll N$, $m \geq K+1$, and moderate noise to observe reliable clustering.


**Further Reading:**
* [**"A Tutorial on Subspace Clustering", René Vidal**](https://www.cis.jhu.edu/~rvidal/publications/SPM-Tutorial-Final.pdf)
* [**"Subspace Clustering", Madalina Ciortan**](https://medium.com/data-science/subspace-clustering-7b884e8fff73)

# 3. MIMO Beamforming: Optimal Subspace via Principal Angles

## 3.1 System Description

Modern wireless systems (e.g. 5G) use **MIMO** (multiple-input multiple-output) channels.

In this system, up to $n$ antennae at the transmitter simultaneously transmit information
that is recived at $m$ antennae on the receiver side.<br>
The channel linearly transforms the transmitted signal, mixing contributions
from all transmit antennas as they propagate to each receiver.

The wireless channel is modeled by a matrix $H \in \mathbb{C}^{m \times n}$,<br>
where each entry describes how the signal from a transmit antenna propagates to a receive antenna.

In [4]:
from panel_mermaid import MermaidDiagram
txt = pn.pane.LaTeX(r"""
<span style="font-size: 16px;">
The transmitter emits an unknown signal $x \in \mathbb{C}^{n}$, and the receiver observes<br><br>

$\qquad
b = H x + \mathrm{noise}
$<br><br>

To send $k$ data streams $s$, the transmitter projects data onto a subspace $U \in \mathrm{Gr}(k, n)$,<br>
and optionally the receiver projects onto a subspace $V \in \mathrm{Gr}(k, m)$.<br>
The effective system becomes<br><br>

$\qquad
b_{\mathrm{eff}} = V^H H U s, \quad s \in \mathbb{C}^k
$<br><br>

The goal is to choose $U$ to align with the dominant directions of $H$,<br>
maximizing received power. Since $U$ is a subspace, this is an optimization problem<br>
on the Grassmannian manifold $\mathrm{Gr}(k, n)$.<br>
</span>
""", renderer='katex')

dg = MermaidDiagram(object="""
flowchart LR
  %% your diagram content
  TX1["📡 TX₁"]
  TX2["📡 TX₂"]
  TX3["📡 TX₃"]
  CH["H ∈ C^{5×3}"]
  RX1["📶 RX₁"]
  RX2["📶 RX₂"]
  RX3["📶 RX₃"]
  RX4["📶 RX₄"]
  RX5["📶 RX₅"]

  subgraph TRANSMITTER["Transmitter"]
    direction TB
    TX1
    TX2
    TX3
  end

  subgraph RECEIVER["Receiver"]
    direction TB
    RX1
    RX2
    RX3
    RX4
    RX5
  end

  TX1 --> CH
  TX2 --> CH
  TX3 --> CH
  CH --> RX1
  CH --> RX2
  CH --> RX3
  CH --> RX4
  CH --> RX5

  style CH fill:#ffffff,stroke:#000,stroke-width:1px
""", width=350)
pn.Row( txt, pn.Spacer(width=20), dg )

Row
    [0] LaTeX(str, renderer='katex')
    [1] Spacer(width=20)
    [2] MermaidDiagram(object='\nflowchart LR\n  %% your..., width=350)

## 3.2 Beamforming as Optimization on the Grassmannian

In practice, the system measures the channel matrix $H$ (e.g., via pilot signals), and selects the number of streams $k$<br>
based on the strength and rank of the channel.

The goal is to find an optimal transmit subspace $U \in \mathrm{Gr}(k, n)$ to shape the transmitted signal<br>
so that it aligns with the strongest directions of the wireless channel $H$ and maximizes received signal energy.

For a fixed channel matrix $H \in \mathbb{C}^{m \times n}$, the received energy is proportional<br>
to the squared Frobenius norm of the projected signal

$\qquad
\Vert H U \Vert_F^2 = \operatorname{Tr}(U^H H^H H U)
$

This leads to the optimization problem
$\qquad\color{darkred}{
\boxed{ \max_{U \in \mathrm{Gr}(k, n)} \operatorname{Tr}(U^H H^H H U)}}
$

Here, $U$ is constrained to have orthonormal columns: $U^H U = I_k$.<br>
Only the subspace $\mathscr{C}(U)$ matters, not the specific matrix.<br>

<details>
<summary><strong>This is a Rayleigh–Ritz-type problem</strong></summary>

The Rayleigh–Ritz principle describes how to find the subspace that maximizes a quadratic form involving a Hermitian matrix.

Given a Hermitian matrix $A \in \mathbb{C}^{n \times n}$, the problem
$$
\max_{U \in \mathbb{C}^{n \times k}, \, U^H U = I} \operatorname{Tr}(U^H A U)
$$
is solved by choosing $U$ whose columns span the subspace of the top $k$ eigenvectors of $A$.

In our case, the matrix $A = H^H H$ captures the energy profile of the channel, and the trace expression $\operatorname{Tr}(U^H H^H H U)$ gives the total transmitted energy observed through the subspace $U$. The Rayleigh–Ritz result tells us that this quantity is maximized when $U$ spans the dominant $k$-dimensional eigenspace of $H^H H$.
</details>
<br>
The optimal subspace is spanned by the top $k$ eigenvectors of the Hermitian matrix $H^\dagger H$.<br>
However, to solve it via gradient-based methods, we must respect the constraint $U \in \mathrm{Gr}(k, n_t),$ i.e.,<br>
updates must remain on the Grassmannian.

However, in many cases the channel changes or is only partially known, and we must solve the problem iteratively.<br>
This requires **gradient-based optimization on the Grassmannian**, where updates stay within the space of $k$-dimensional subspaces.

We are optimizing over subspaces $U \in \mathrm{Gr}(k, n)$, not over arbitrary matrices.<br>
This requires that updates remain on the manifold — i.e., the span of $U$ must stay orthonormal and $U^H U = I$.

As described in [**GrassmannianGeodesicsOptimization.ipynb**](GrassmannianGeodesicsOptimization.ipynb),<br>
this is done by computing gradients in the tangent space $T_U \mathrm{Gr}(k, n)$<br>
and using retractions or projection steps to stay on the manifold.

____
To complete the formulation of our problem, we wish to maximize the objective function

$\qquad
f(U) = \operatorname{Tr}(U^H H^H H U)\;\;
$
over $U \in \mathrm{Gr}(k, n)$.

The Euclidean gradient of $f$ is
$\qquad
\nabla f(U) = 2 H^H H U
$

The Riemannian (Grassmannian) gradient is its projection onto the tangent space at $U$, which we obtain by subtracting the component along $U$<br>
$\qquad
\operatorname{grad} f(U) = \nabla f(U) - U (U^H \nabla f(U))
$

This expression follows directly from the projection formula introduced in
[**GrassmannianGeodesicsOptimization.ipynb**](GrassmannianGeodesicsOptimization.ipynb),<br>
and uses basic matrix derivative identities discussed in
[**MatrixDerivatives.ipynb**](MatrixDerivatives.ipynb).

We now apply this gradient in a manifold-aware optimization loop to find the optimal beamformer.

## 3.3 Demo: Grassmannian Beamforming

The demo simulates a MIMO system where a measured channel matrix $H \in \mathbb{C}^{m \times n}$ is randomly generated.

The user selects:
- The number of transmit antennas $n$
- The number of receive antennas $m$
- The number of transmit streams $k$
- The number of optimization steps

The system:
- Initializes a random beamformer $U \in \mathrm{Gr}(k, n)$
- Applies gradient ascent on the Grassmannian to maximize $\operatorname{Tr}(U^H H^H H U)$
- Retracts after each step to ensure $U$ remains on $\mathrm{Gr}(k, n)$

The underlying optimization method was implemented in
[**GrassmannianGeodesicsOptimization.ipynb**](GrassmannianGeodesicsOptimization.ipynb).<br>
This demo builds on that code and applies it to a concrete communication scenario.

The demo shows:
- The convergence of the received power $\| H U \|_F^2$ over iterations
- A 2D projection of the subspace trajectory (via PCA)

This illustrates how geometry-aware optimization improves alignment with the channel and increases received energy.

In [5]:
class BeamformingDemo(pn.viewable.Viewer):
    channel_type = param.ObjectSelector(default="Rayleigh", 
                                    objects=["Rayleigh", "Low-Rank", "Correlated"],
                                    label="Channel Model")

    n = param.Integer(6, bounds=(2, 20), label="Transmit Antennas (n)")
    m = param.Integer(6, bounds=(2, 20), label="Receive Antennas (m)")
    k = param.Integer(2, bounds=(1, 10), label="Streams (k)")
    steps = param.Integer(20, bounds=(1, 100), label="Gradient Steps")
    lr = param.Number(0.1, bounds=(0.01, 1.0), label="10*Step Size")

    def __init__(self, **params):
        super().__init__(**params)
        self._validate_k()

    @param.depends('n', 'k', watch=True)
    def _validate_k(self):
        self.param.k.bounds = (1, self.n)
        if self.k > self.n:
            self.k = self.n

    def generate_channel(self, m, n):
        if self.channel_type == "Rayleigh":
            return np.random.randn(m, n) + 1j * np.random.randn(m, n)

        elif self.channel_type == "Low-Rank":
            r = min(m, n) // 2  # force rank-deficiency
            A = np.random.randn(m, r) + 1j * np.random.randn(m, r)
            B = np.random.randn(r, n) + 1j * np.random.randn(r, n)
            return A @ B

        elif self.channel_type == "Correlated":
            # Basic Kronecker model with Tx and Rx covariance
            Rt = np.diag(np.linspace(1, 0.5, n))
            Rr = np.diag(np.linspace(1, 0.5, m))
            W = np.random.randn(m, n) + 1j * np.random.randn(m, n)
            return Rr @ W @ Rt

    def random_unitary_subspace(self, n, k):
        Q, _ = np.linalg.qr(np.random.randn(n, k) + 1j * np.random.randn(n, k))
        return Q

    def hermitian_gradient(self, H, U):
        return 2 * H.conj().T @ (H @ U)

    def project_to_tangent(self, U, G):
        return G - U @ (U.conj().T @ G)

    def retract(self, U):
        Q, _ = np.linalg.qr(U)
        return Q

    def run_optimization(self, H, U_init, steps, lr):
        powers = []
        subspaces = []

        U = U_init.copy()
        for _ in range(steps):
            G = self.hermitian_gradient(H, U)
            G_proj = self.project_to_tangent(U, G)
            U = self.retract(U + lr * G_proj)
            powers.append(np.linalg.norm(H @ U, ord='fro')**2)
            subspaces.append(U[:, 0].real)  # First beamforming vector (real part)
        return powers, np.array(subspaces)

    @param.depends('n', 'm', 'k', 'steps', 'lr')
    def view(self):
        n, m, k, steps, lr = self.n, self.m, self.k, self.steps, self.lr

        H      = self.generate_channel(m, n)
        rank_H = np.linalg.matrix_rank(H, tol=1e-10)
        U0     = self.random_unitary_subspace(n, k)

        powers, vectors = self.run_optimization(H, U0, steps, 0.01*lr)

        # Plot received power over iterations
        power_curve = hv.Curve((np.arange(1, steps+1), powers), 'Step', 'Received Power').opts(
            height=300, width=400, line_width=2, title=f"Received Power over Iterations, rank(H) ={rank_H}"
        )

        # PCA projection of subspace trajectory
        if len(vectors) >= 2:
            coords = PCA(n_components=2).fit_transform(vectors)
            steps = np.arange(len(coords))

            trajectory = hv.Points(
                (coords[:, 0], coords[:, 1], steps),
                kdims=['PC1', 'PC2'],
                vdims='Step'
            ).opts(
                cmap='cividis',
                color='Step',
                size=6,
                marker='circle',
                width=400,
                height=300,
                title="Subspace Trajectory (PCA)",
                tools=['hover']
            )
            path = hv.Path([coords], kdims=['PC1', 'PC2'])

            trajectory_plot = (path * trajectory)
        else:
            trajectory_plot = hv.Text(0, 0, "Insufficient steps").opts(width=400, height=300)

        return pn.Row(power_curve, trajectory_plot)

    def __panel__(self):
        controls = pn.Param(self, show_name=False, width=300)
        return pn.Column(pn.Row(controls, self.view))

BeamformingDemo().servable()

Column
    [0] Row
        [0] Param(BeamformingDemo, show_name=False, width=300)
        [1] ParamMethod(method, _pane=Row, defer_load=False)

**Geometry-aware Beamforming Optimization**<br>
The **left plot** shows how received power $\Vert H U \Vert_F^2$ increases over gradient steps.<br>
The **right plot** shows the trajectory of the beamforming subspace (first vector of $U$) projected to 2D via PCA.<br>
Values are colored from dark to light by step number.

Together, they illustrate how optimization on $\mathrm{Gr}(k, n)$ gradually aligns the transmit subspace with the dominant directions of the channel $H$.

# 4. Grassmannian Interpolation of Subspaces

In the previous section, we computed an optimal transmit subspace $U \in \mathrm{Gr}(k, n)$ for a measured channel matrix $H$.

In practice, the channel evolves over time. When a new channel matrix $H_{new}$ is measured, the system computes a new optimal precoder $U_{new}$.<br>
$\qquad$ The simplest approach is to switch instantly from $U$ to $U_{new}$.

However, abrupt switching may cause problems:
- The transmit beam pattern may change discontinuously, stressing power amplifiers and violating spectral masks.
- The receiver may not track the change instantly, causing packet errors.
- In multi-user MIMO, sudden precoder changes may disrupt fairness and stability.

A better approach is to **smoothly interpolate** between the current precoder $U$ and the new precoder $U_{new}$,<br>
$\qquad$ evolving the transmit subspace in a controlled way.<br>
$\qquad$ Since the space of $k$-dimensional subspaces is the Grassmannian manifold $\mathrm{Gr}(k, n)$,<br>
$\qquad$ the natural tool is **Grassmannian geodesic interpolation**.

This ensures that the interpolated precoder $U(t)$ remains a valid subspace at every intermediate step,<br>
$\qquad$ avoiding the artifacts introduced by naive linear interpolation.


In [6]:
class GrassmannInterpolationDemo(pn.viewable.Viewer):
    n             = param.Integer( 8, bounds=(2, 20), label="Transmit Antennas")
    m             = param.Integer( 8, bounds=(2, 20), label="Receive Antennas")
    k             = param.Integer( 2, bounds=(1, 20), label="Streams")
    snr           = param.Number( 10, bounds=(0, 30), label="SNR (dB)", precedence=-1)
    steps         = param.Integer(50, bounds=(10, 200), label="Interpolation Steps")
    max_angle_deg = param.Number( 60, bounds=(0, 90), label="Max Principal Angle (deg)")

    def __init__(self, **params):
        super().__init__(**params)
        self._reset_system()

    @param.depends('n', 'm', 'k', watch=True)
    def _validate_params(self):
        # Enforce k ≤ n
        if self.k > self.n:
            self.k = self.n
        self.param.k.bounds = (1, self.n-1)

        # Enforce n ≤ m
        if self.n > self.m:
            self.n = self.m
        self.param.n.bounds = (2, self.m)

    @param.depends('n', 'm', 'k', 'snr', 'max_angle_deg', watch=True)
    def _reset_system(self):
        self._validate_params()
        self.U0, self.U1 = self._random_unitary_subspaces(self.n, self.k, self.max_angle_deg)
        self.H = self._generate_channel(self.m, self.n)

    def _random_unitary_subspaces(self, n, k, max_angle_deg):
        # Orthonormal basis Q0
        Q_big, _ = np.linalg.qr(np.random.randn(n, n))
        Q0 = Q_big[:, :k]

        # Build orthogonal complement of Q0
        if n - k > 0:
            Q_orth, _ = np.linalg.qr(np.random.randn(n, n - k))
        else:
            Q_orth = np.zeros((n, 0))

        # Desired principal angles
        theta_max = np.deg2rad(max_angle_deg)
        theta = np.zeros(k)
        theta[0] = theta_max
        #if k > 1:
        #    theta[1:] = 0.0

        # Build U1 according to theory:
        # U1 = Q0 * diag(cosθ) + Q_perp * diag(sinθ)
        U1 = np.zeros((n, k), dtype=complex)
        cos_theta = np.cos(theta)
        sin_theta = np.sin(theta)

        for i in range(k):
            # Q0[:, i] is always there
            vec = cos_theta[i] * Q0[:, i]
            # Add orthogonal component if available
            if i < Q_orth.shape[1]:
                vec += sin_theta[i] * Q_orth[:, i]
            # Store column
            U1[:, i] = vec

        # Final orthogonalization to ensure U1 is orthonormal
        Q1, _ = np.linalg.qr(U1)

        # Optional debug print:
        #print("Principal angles U0 → U1:", np.round(np.degrees(self._principal_angles(Q0, Q1)), 2))

        return Q0, Q1

    def _generate_channel(self, m, n):
        return np.random.randn(m, n) + 1j * np.random.randn(m, n)

    def _linear_interpolate(self, t):
        U_t = (1 - t) * self.U0 + t * self.U1
        Q, _ = np.linalg.qr(U_t)
        return Q

    def _geodesic_interpolate(self, t):
        M = self.U0.conj().T @ self.U1
        U, s, Vh = np.linalg.svd(M, full_matrices=False)
        theta = np.arccos(np.clip(s, 0, 1))

        sin_theta = np.sin(theta)
        sin_theta[sin_theta < 1e-12] = 1e-12

        term1 = self.U0 @ U @ np.diag(np.cos(theta * t))
        term2 = (self.U1 - self.U0 @ U @ np.diag(s) @ Vh) @ Vh.conj().T
        term2 = term2 @ np.diag(np.sin(theta * t) / sin_theta)
        U_t = term1 + term2

        Q, _ = np.linalg.qr(U_t)
        return Q

    def _principal_angles(self, U, V):
        M = U.conj().T @ V
        s = np.linalg.svd(M, compute_uv=False)
        return np.arccos(np.clip(s, 0, 1))

    def _calculate_metrics(self, method):
        t_values = np.linspace(0, 1, self.steps)
        powers = []
        angles = []

        for t in t_values:
            if method == 'linear':
                U_t = self._linear_interpolate(t)
            else:
                U_t = self._geodesic_interpolate(t)

            powers.append(np.linalg.norm(self.H @ U_t, 'fro')**2)

            pa = np.sort(self._principal_angles(U_t, self.U1))[::-1]
            angles.append(pa[0])

        return t_values, np.array(powers), np.rad2deg(np.array(angles))

    @param.depends('n', 'm', 'k', 'snr', 'steps', 'max_angle_deg')
    def view(self):
        t_lin, power_lin, angles_lin = self._calculate_metrics('linear')
        t_geo, power_geo, angles_geo = self._calculate_metrics('geodesic')
        foo = np.max(angles_geo)

        power_plot = hv.Curve((t_lin, power_lin), 't', 'Received Power', label='Linear').opts(color='blue') * \
                     hv.Curve((t_geo, power_geo), 't', 'Received Power', label='Geodesic').opts(color='red')
        power_plot.opts(
            title="Received Power Comparison",
            width=500, height=300,
            legend_position='top'
        )

        angle_plot = hv.Curve((t_lin, angles_lin), 't', 'Angle (deg)', label='Linear').opts(color='blue') * \
                     hv.Curve((t_geo, angles_geo), 't', 'Angle (deg)', label='Geodesic').opts(color='red')
        angle_plot.opts(
            title=f"Largest Principal Angle to Target Subspace ",
            width=500, height=300,
            legend_position='top'
        )

        return pn.Row(power_plot, angle_plot)

    def __panel__(self):
        return pn.Column(
            self.view,
            pn.Param(self, parameters=['n', 'm', 'k', 'snr', 'steps', 'max_angle_deg'], show_name=False)
        )

demo = GrassmannInterpolationDemo()
demo.servable()

Column
    [0] ParamMethod(method, _pane=Row, defer_load=False)
    [1] Param(GrassmannInterpolationDemo, show_name=False)

____
Both interpolations reach the same endpoint at $t=1$, but follow different trajectories.<br>
This is reflected in the evolution of both received power and the principal angles<br>
between the current subspace $\mathscr{C}(U(t))$ and the target subspace $\mathscr{C}(U_{new})$.

The goal of this demo is not to suggest that Grassmannian interpolation should be used in practical MIMO systems.<br>
Rather, it is intended to help visualize how Grassmannian geometry affects the behavior of interpolated subspaces,<br>
and to contrast this with the simpler linear projection interpolation.

____
In an actual MIMO system, precoder interpolation would typically be applied when the channel varies slowly.<br>
In that case, principal angles between successive precoders would be small, and both interpolation methods would perform similarly.<br>
**Large principal angles, as shown here, are used to illustrate the geometric behavior of Grassmannian interpolation.**

In the principal angles plot, observe that some angles remain large for most of the interpolation,
then drop sharply near $t=1$.<br>
This occurs because Grassmannian interpolation must exactly match the target subspace $\mathscr{C}(U_{new})$ at $t=1$.<br>
Large initial principal angles imply that the corresponding directions must rotate rapidly near the end of the path.<br>
This geometric effect explains the ``knee''—the sharp change in slope—often seen in the received power curve near $t=1$.

This example highlights that while Grassmannian interpolation provides a geometrically optimal path between subspaces,<br>
engineering objectives such as received power may favor simpler approaches:<br>
understanding this tradeoff is precisely the value of the tools we developed in this notebook arc.

# 5. Take Away

This notebook concludes the study of Grassmannian geometry, from principal angles to applications.

Subspaces frequently model data and systems.
Key problems include
* **identifying relevant subspaces** from data,
* **optimizing subspaces** for specific objectives, and
* **interpolating between subspaces**
as illustrated by the examples provided in this notebook.

Grassmannian geodesics define mathematically sound paths for interpolation.<br>
Such tools also enable analysis of tradeoffs between geometric optimality and application performance, as illustrated in the final MIMO example.

Principal angles, Grassmannian geodesics, and optimization on $\mathrm{Gr}(k,n)$ provide a rigorous framework for these tasks.<br>
They are valuable both when the Grassmannian is used explicitly and when alternative methods are employed.